In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

dataset_dir = "archive/train_test"

# Preprocesses an image before it is given to the model to ensure consistency
# across all images that the model is learning off of.
def preprocess_image(image_path):
    # Load the image and convert it to RGB colour space; 
    # we want all images to be in same colour space for consistency.
    with Image.open(image_path) as image:
        image = image.convert("RGB")
    # Resize the image to a standard size of 256 x 256.
    image = image.resize((256, 256))
    # Convert the image to an array. Normalize its pixel values to be between 0 and 1.
    # Since most images in the training/test dataset are jpegs and in RGB, assume that
    # we can divide each pixel by 255 to normalize them to be between 0 and 1.
    preprocessed_image = np.array(image) / 255.0
    # Return the preprocessed image.
    return preprocessed_image

def is_authentic(filename):
    prefix = filename[:2]
    return (prefix == "au" or prefix == "Au")

images = []
labels = []

for filename in os.listdir(dataset_dir):
    if filename.startswith(".DS_Store"):
        continue
    file_path = os.path.join(dataset_dir, filename)
    preprocessed_image = preprocess_image(file_path)
    images.append(preprocessed_image)
    label = is_authentic(filename)
    labels.append(label)

images = np.array(images)
labels = np.array(labels)

data_augmentation = ImageDataGenerator(
    rotation_range = 25,
    height_shift_range = 0.25,
    width_shift_range = 0.25,
    vertical_flip = True,
    horizontal_flip = True,
    zoom_range = 0.25
)

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the model architecture
base = tf.keras.applications.EfficientNetV2S(include_top=False, input_shape=train_images.shape[1:])

x = tf.keras.layers.GlobalAveragePooling2D()(base.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=base.input, outputs=output)

# Compile and train the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy', metrics=['accuracy'])

results = model.fit(data_augmentation.flow(train_images, train_labels, batch_size=32), epochs=20,
                    steps_per_epoch=len(train_images) // 32,
                    validation_data=(test_images, test_labels))

Metal device set to: Apple M1 Max
Epoch 1/20


2023-04-13 11:12:54.839768: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


196/196 [==============================] - 155s 641ms/step - loss: 0.5681 - accuracy: 0.6985 - val_loss: 0.4698 - val_accuracy: 0.7703
Epoch 2/20
196/196 [==============================] - 92s 458ms/step - loss: 0.4770 - accuracy: 0.7787 - val_loss: 0.4493 - val_accuracy: 0.7665
Epoch 3/20
196/196 [==============================] - 95s 483ms/step - loss: 0.4209 - accuracy: 0.8070 - val_loss: 0.4360 - val_accuracy: 0.7944
Epoch 4/20
196/196 [==============================] - 91s 466ms/step - loss: 0.3844 - accuracy: 0.8243 - val_loss: 0.4095 - val_accuracy: 0.7938
Epoch 5/20
196/196 [==============================] - 90s 459ms/step - loss: 0.3545 - accuracy: 0.8397 - val_loss: 0.3905 - val_accuracy: 0.8020
Epoch 6/20
196/196 [==============================] - 90s 460ms/step - loss: 0.3308 - accuracy: 0.8456 - val_loss: 0.4110 - val_accuracy: 0.7779
Epoch 7/20
196/196 [==============================] - 92s 469ms/step - loss: 0.3203 - accuracy: 0.8508 - val_loss: 0.4107 - val_accuracy: 0.

In [2]:
# Convert and save the model as a TensorFlow Lite file
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('manipulation_detector_config_3.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/x9/lnqw3ctd65g1t8sjc2w_k5hm0000gn/T/tmp7zxtz85n/assets


INFO:tensorflow:Assets written to: /var/folders/x9/lnqw3ctd65g1t8sjc2w_k5hm0000gn/T/tmp7zxtz85n/assets
2023-04-13 11:46:07.388136: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-04-13 11:46:07.388377: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.


In [3]:
# Evaluate the model on the testing set and print the accuracy
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test accuracy:", test_acc)

50/50 [==============================] - 5s 91ms/step - loss: 0.6248 - accuracy: 0.7760
Test accuracy: 0.7760152220726013
